In [ ]:
import sys
sys.path.append("../..")
import torch
from lru.architectures import DLRU, DLRUConfig
import matplotlib.pyplot as plt
import torch 
import torchid.metrics # pip install pytorch-ident
import nonlinear_benchmarks
import math

In [ ]:
%matplotlib widget

In [ ]:
ckpt = torch.load("ckpt_no_reg.pt", map_location="cpu")
#ckpt = torch.load("ckpt_hankel_reg.pt", map_location="cpu")

In [ ]:
cfg = ckpt["cfg"]
config = DLRUConfig(d_model=cfg.d_model, d_state=cfg.d_state, n_layers=cfg.n_layers, ff=cfg.ff, max_phase=math.pi)
model = DLRU(1, 1, config)

In [ ]:
train_val, test = nonlinear_benchmarks.WienerHammerBenchMark()
sampling_time = train_val.sampling_time #in seconds
u_train, y_train = train_val #or train_val.u, train_val.y
u_test, y_test = test        #or test.u,      test.y

u_train = u_train.reshape(-1, 1)
y_train = y_train.reshape(-1, 1)
u_test = u_test.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)

In [ ]:
ckpt["cfg"]

In [ ]:
scaler_u = ckpt["scaler_u"]
scaler_y = ckpt["scaler_y"]

In [ ]:

config = DWNConfig(d_model=5, d_state=16, n_layers=3, ff="MLP")
model = DWN(1, 1, config)
model.load_state_dict(ckpt["model"])

In [ ]:
sum(map(torch.numel, model.parameters()))

In [ ]:
ut = torch.tensor(scaler_u.transform(u_test)).unsqueeze(0).float()
with torch.no_grad():
    y_test_hat = model(ut, mode="loop").squeeze(0).to("cpu").numpy()

y_test_hat = scaler_y.inverse_transform(y_test_hat)

In [ ]:
plt.figure()
plt.plot(ckpt["LOSS"])

plt.figure()
#plt.ion()
plt.plot(y_test, "k")
plt.plot(y_test_hat, "b")
plt.show()
plt.savefig("fit.pdf")

In [ ]:
rmse = torchid.metrics.rmse(y_test, y_test_hat)[0]*1000
fit = torchid.metrics.fit_index(y_test, y_test_hat)[0]

print(f"{rmse=} mV\n{fit=}")

In [ ]:
lambdas, B, C, D = model.blocks[2].lru.ss_params()
lambdas = lambdas.detach()

In [ ]:
import numpy as np
#plt.figure(figsize=(2*config.n_layers+2, 2))
fig, ax = plt.subplots(1, config.n_layers, figsize=(10,5))
t = np.linspace(0, 2*np.pi, 1000)
for idx, layer in enumerate(model.blocks):#range(config.n_layers):
    lambdas, B, C, D = model.blocks[idx].lru.ss_params()
    lambdas = lambdas.detach()
    ax[idx].plot(np.cos(t), np.sin(t), "-k")
    ax[idx].plot(lambdas.real, lambdas.imag, "k*")#lambdas.real
    ax[idx].plot(lambdas.real, -lambdas.imag, "k*")#lambdas.real
    ax[idx].set_box_aspect(1)

In [ ]:
matlab_dict = {}
for idx, block in enumerate(model.blocks):
    A, B, C, D = block.lru.ss_real_matrices()
    matlab_dict[f"layer{idx}"] = {"A": A, "B": B, "C": C, "D": D}

In [ ]:
import scipy.io as sio
sio.savemat("lti_matrices.mat", matlab_dict)